In [1]:
#Importamos las librerías
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('once')


In [2]:
df=pd.read_csv('data/bikes_regis_estand_encod.csv',index_col=0)
df.head(3)

,registro,clima,temperatura,sens_termica,humedad,viento,registrados,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,-0.828948,-0.678635,1.295131,-0.376545,654,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,-0.722312,-0.740034,0.507407,0.882110,670,2018-01-02,NaN,0,1,1,1,0,0
2,3,1,-1.642047,-1.746977,-1.354487,0.861132,1229,2018-01-03,NaN,0,1,2,1,0,0


In [3]:
df.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'registrados', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [4]:
X=df.drop(['registrados'],axis=1)
y=df['registrados']

In [5]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [6]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [7]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

19

In [8]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [9]:
def obtener_metricas(y_test_m, y_train_m, y_predict_test_m, y_predict_train_m, modelo):
    # En un diccionario, calcular las métricas de train y test
    resultados_metricas = {
        'MAE': [mean_absolute_error(y_test_m, y_predict_test_m), mean_absolute_error(y_train_m, y_predict_train_m)],
        'MSE': [mean_squared_error(y_test_m, y_predict_test_m), mean_squared_error(y_train_m, y_predict_train_m)],
        'RMSE': [np.sqrt(mean_squared_error(y_test_m, y_predict_test_m)), np.sqrt(mean_squared_error(y_train_m, y_predict_train_m))],
        'R2':  [r2_score(y_test_m, y_predict_test_m), r2_score(y_train_m, y_predict_train_m)],
        "set": ["test", "train"], 
        "modelo": [modelo, modelo]}

    # Pasamos a dataframe el diccionario para su mejor visualizacion
    df_resultados_metricas = pd.DataFrame(resultados_metricas)

    return df_resultados_metricas

In [10]:
df_resultados1= obtener_metricas(y_test,y_train,y_pred_test1,y_pred_train1,'Decision Tree Codif y Estand')
df_resultados1

,MAE,MSE,RMSE,R2,set,modelo
0,502.294521,648977.075342,805.591134,0.699349,test,Decision Tree Codif y Estand
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree Codif y Estand


In [11]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,5,9], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [12]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [13]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 5, 9], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [14]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=9, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [15]:
y_pred_test_dt= mejor_modelo.predict(x_test)
y_pred_train_dt= mejor_modelo.predict(x_train)

In [16]:
df_resultados= obtener_metricas(y_test,y_train,y_pred_test_dt,y_pred_train_dt,'Decision Tree Codif y Estand')
df_resultados

,MAE,MSE,RMSE,R2,set,modelo
0,608.627631,684698.674732,827.465211,0.682800,test,Decision Tree Codif y Estand
1,511.512893,482579.645123,694.679527,0.806432,train,Decision Tree Codif y Estand


In [17]:
max_features= np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [18]:
max_depth = arbol.tree_.max_depth 
max_depth

19

Pruebas de modificación de parametros


In [19]:
param1 = {"max_depth": [5, 7, 8], 
         "max_features": [1,2,3], 
         "min_samples_split": [10, 75, 150], 
         "min_samples_leaf": [10, 75, 150]}

In [20]:
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param1, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [21]:
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [5, 7, 8], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 75, 150],
                         'min_samples_split': [10, 75, 150]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [22]:
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=8, max_features=3, min_samples_leaf=10,
                      min_samples_split=75)

In [23]:
y_pred_test2= mejor_modelo.predict(x_test)
y_pred_train2= mejor_modelo.predict(x_train)



In [24]:
df_resultados2= obtener_metricas(y_test,y_train,y_pred_test2,y_pred_train2,'Decision Tree Codif y Estand')
df_resultados2

,MAE,MSE,RMSE,R2,set,modelo
0,806.246466,1.027240e+06,1013.528350,0.524111,test,Decision Tree Codif y Estand
1,839.332789,1.073182e+06,1035.944901,0.569536,train,Decision Tree Codif y Estand


In [25]:
df['registrados'].max()

6946

In [72]:
df['registrados'].min()

20

Probamos solo con registrados codificadas:

In [27]:
df2 = pd.read_csv('data/bikes_regis_encod.csv', index_col=0)

In [28]:
df2.head(2)

,registro,clima,temperatura,sens_termica,humedad,viento,registrados,fecha_nueva,festividad,festividad_1,mes_bueno,dia_semana_nuevo,no_laboral_nuevo,año_map,estacion_map
0,1,2,14.1,18.2,81.0,10.7,654,2018-01-01,New Year's Day,1,1,0,0,0,0
1,2,2,14.9,17.7,70.0,16.7,670,2018-01-02,NaN,0,1,1,1,0,0


In [29]:
df2.drop(['registro','fecha_nueva','festividad'], axis=1,inplace=True)
df2.columns

Index(['clima', 'temperatura', 'sens_termica', 'humedad', 'viento',
       'registrados', 'festividad_1', 'mes_bueno', 'dia_semana_nuevo',
       'no_laboral_nuevo', 'año_map', 'estacion_map'],
      dtype='object')

In [30]:
X=df2.drop(['registrados'],axis=1)
y=df2['registrados']

In [31]:
# Separamos en train y test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Creamos objeto del modelo
arbol=DecisionTreeRegressor(random_state=0)
#Ajustamos el modelo
arbol.fit(x_train,y_train)

DecisionTreeRegressor(random_state=0)

In [32]:
# Obtenemos la raíz cuadrada del número de variables predictoras de nuestro árbol
max_features = np.sqrt(len(x_train.columns))
max_features

3.3166247903554

In [33]:
# Obtenemos la profundidad del árbol representado
arbol.tree_.max_depth

19

In [34]:
# Predicción para los datos de entrenamiento
y_pred_train1 = arbol.predict(x_train)
# Predicción para los datos de test
y_pred_test1 = arbol.predict(x_test)

In [36]:
df_resultados3= obtener_metricas(y_test,y_train,y_pred_test1,y_pred_train1,'Decision Tree Codif')
df_resultados3

,MAE,MSE,RMSE,R2,set,modelo
0,502.849315,649094.136986,805.663787,0.699295,test,Decision Tree Codif
1,0.000000,0.000000,0.000000,1.000000,train,Decision Tree Codif


In [37]:
# Definir diccionario con los hiperparámetros a modificar
# min_samples_split: número mínimo de observaciones que debe tener un nodo para dividirse, por defecto [10, 50, 100]
# min_samples_leaf: número mínimo de observaciones que debe tener cada uno de los nodos hijos para que se produzca la division, por defecto [10, 50, 100]
param = {"max_depth": [2,5,9], "max_features": [1,2,3], "min_samples_split": [10, 150, 300], "min_samples_leaf": [10,150,300]}

In [38]:
# Ajustar el modelo con GridSearchCV
# cv = 10 --> número de veces que hacemos la validación
# verbose = - 1 --> para no printear ningún mensaje en la pantalla
grid = GridSearchCV(estimator=DecisionTreeRegressor(), param_grid=param, cv=10, verbose=-1,
                        return_train_score=True, scoring="neg_mean_squared_error")

In [39]:
# Ajustar el modelo
grid.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [2, 5, 9], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 150, 300],
                         'min_samples_split': [10, 150, 300]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [40]:
# El método nos dice cuál es el mejor modelo,
mejor_modelo = grid.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=9, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [41]:
y_pred_test_dt= mejor_modelo.predict(x_test)
y_pred_train_dt= mejor_modelo.predict(x_train)

In [42]:
df_resultados4= obtener_metricas(y_test,y_train,y_pred_test_dt,y_pred_train_dt,'Decision Tree Codif II')
df_resultados4

,MAE,MSE,RMSE,R2,set,modelo
0,641.321307,689987.871179,830.655086,0.680350,test,Decision Tree Codif II
1,560.395852,583714.081996,764.011834,0.765866,train,Decision Tree Codif II


Siguientes pasos:
- probar DT con solo las codificadas (sin estandarizar)
- realizar random forest
- comparar métricas